### Visualization without Echo

In [4]:
import numpy as np
import qutip as qt
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider
from system_single import AtomCavitySystem


def run_simulation(gamma, kappa, beta):
    # Create system
    system = AtomCavitySystem(N=10, g=0, w0=(1.0 * 2 * np.pi),
                              gamma=gamma, kappa=kappa, beta=beta)

    # Build Hamiltonian
    H = system.create_hamiltonian()

    # Initial state |0⟩_cavity ⊗ |+⟩_atom
    psi11 = system.create_initial_state('1', 'g')

    # Collapse operators
    c_ops = system.create_collapse_operators(leaking=True, decay=True, dephasing=True)

    # Expectation values
    e_ops = system.create_expectation_values(
        number_atom=True,
        number_cavity=True,
        coherence_ge=True,
        coherence_eg=True
    )

    # Time evolution
    times = np.linspace(0, 20, 500)
    result = qt.mesolve(H, psi11, times, c_ops, e_ops)

    # Plot
    plt.figure(figsize=(8, 5))
    plt.plot(times, result.expect[0], label="⟨n_atom⟩")
    plt.plot(times, result.expect[1], label="⟨n_cavity⟩")
    plt.plot(times, np.abs(result.expect[2]), label="|coherence (atom)|")
    plt.legend()
    plt.xlabel("Time")
    plt.ylabel("Expectation value")
    plt.grid(True)
    plt.show()

# Create interactive sliders
interact(run_simulation,
         gamma=FloatSlider(min=0, max=0.5, step=0.01, value=0, description="Leaking γ"),
         kappa=FloatSlider(min=0, max=0.5, step=0.01, value=0, description="Decay κ"),
         beta=FloatSlider(min=0, max=0.5, step=0.01, value=0, description="Dephasing β"));


interactive(children=(FloatSlider(value=0.0, description='Leaking γ', max=0.5, step=0.01), FloatSlider(value=0…

In [5]:
import numpy as np
import qutip as qt
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider
from system_single import AtomCavitySystem


def run_simulation(gamma, kappa, beta):
    # Create system
    system = AtomCavitySystem(N=10, g=(0.1 * 2 * np.pi), w0=(1.0 * 2 * np.pi),
                              gamma=gamma, kappa=kappa, beta=beta)

    # Build Hamiltonian
    H = system.create_hamiltonian()

    # Initial state |0⟩_cavity ⊗ |+⟩_atom
    psi11 = system.create_initial_state('1', 'g')

    # Collapse operators
    c_ops = system.create_collapse_operators(leaking=True, decay=True, dephasing=True)

    # Expectation values
    e_ops = system.create_expectation_values(
        number_atom=True,
        number_cavity=True,
        coherence_ge=True,
        coherence_eg=True
    )

    # Time evolution
    times = np.linspace(0, 20, 500)
    result = qt.mesolve(H, psi11, times, c_ops, e_ops)

    # Plot
    plt.figure(figsize=(8, 5))
    plt.plot(times, result.expect[0], label="⟨n_atom⟩")
    plt.plot(times, result.expect[1], label="⟨n_cavity⟩")
    plt.plot(times, np.abs(result.expect[2]), label="|coherence (atom)|")
    plt.legend()
    plt.xlabel("Time")
    plt.ylabel("Expectation value")
    plt.grid(True)
    plt.show()

# Create interactive sliders
interact(run_simulation,
         gamma=FloatSlider(min=0, max=0.5, step=0.01, value=0, description="Leaking γ"),
         kappa=FloatSlider(min=0, max=0.5, step=0.01, value=0, description="Decay κ"),
         beta=FloatSlider(min=0, max=0.5, step=0.01, value=0, description="Dephasing β"));


interactive(children=(FloatSlider(value=0.0, description='Leaking γ', max=0.5, step=0.01), FloatSlider(value=0…

### Adding an Echo

In [ ]:
import numpy as np
import qutip as qt
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider

from system_single import AtomCavitySystem


def run_simulation(gamma, kappa, sigma, t_echo):
    num_realizations=20
    # Create system (no Lindblad dephasing here, only leakage/decay)
    system = AtomCavitySystem(N=10, g=0, w0=(1.0 * 2 * np.pi),
                              gamma=gamma, kappa=kappa, beta=0.0)

    H = system.create_hamiltonian()
    psi0 = system.create_initial_state('1', '+')

    # Collapse operators: allow leakage/decay, but NOT sz dephasing
    c_ops = system.create_collapse_operators(leaking=(gamma > 0),
                                             decay=(kappa > 0),
                                             dephasing=False)

    # Times
    t_max = 100
    times1 = np.linspace(0, t_echo, 100)
    times2 = np.linspace(t_echo, t_max, 200)
    times_full = np.concatenate([times1, times2])

    # Pi pulse
    X_pi = qt.tensor(qt.qeye(system.N), qt.sigmax())

    def evolve_with_static_noise(delta_phi):
        # Add quasi-static noise as z-shift
        H_noise = H + 0.5 * delta_phi * qt.tensor(qt.qeye(system.N), qt.sigmaz())

        result1 = qt.mesolve(H_noise, psi0, times1, c_ops, [], options=qt.Options(store_states=True))
        rho_mid = result1.states[-1]
        rho_mid_pulsed = X_pi * rho_mid * X_pi.dag()
        result2 = qt.mesolve(H_noise, rho_mid_pulsed, times2, c_ops, [], options=qt.Options(store_states=True))

        return result1.states + result2.states

    # Run multiple realizations
    all_states = []
    for _ in range(num_realizations):
        delta_phi = np.random.normal(0, sigma)
        states = evolve_with_static_noise(delta_phi)
        all_states.append(states)

    # Average states
    avg_states = []
    for i in range(len(times_full)):
        rho_avg = sum([all_states[r][i] for r in range(num_realizations)]) / num_realizations
        avg_states.append(rho_avg)

    # Compute observables
    n_atom = [qt.expect(system.sm.dag() * system.sm, rho) for rho in avg_states]
    n_cavity = [qt.expect(system.a.dag() * system.a, rho) for rho in avg_states]
    coh = [abs(qt.expect(system.coh_ge, rho)) for rho in avg_states]

    # Plot
    plt.figure(figsize=(8, 5))
    plt.plot(times_full, n_atom, label="⟨n_atom⟩")
    plt.plot(times_full, n_cavity, label="⟨n_cavity⟩")
    plt.plot(times_full, coh, label="|coherence atom|")
    plt.legend()
    plt.xlabel("Time")
    plt.ylabel("Expectation value")
    plt.title("Spin Echo with Quasi-Static Noise")
    plt.grid(True)
    plt.show()


# Interactive sliders
interact(run_simulation,
         gamma=FloatSlider(min=0, max=0.5, step=0.01, value=0.0, description="Leaking γ"),
         kappa=FloatSlider(min=0, max=0.5, step=0.01, value=0.0, description="Decay κ"),
         sigma=FloatSlider(min=0.0, max=1.0, step=0.05, value=0.2, description="Noise σ"),
         t_echo=FloatSlider(min=0.1, max=60, step=0.5, value=30, description="Echo time"));


interactive(children=(FloatSlider(value=0.0, description='Leaking γ', max=0.5, step=0.01), FloatSlider(value=0…

### Visualization of the atom state on the Bloch sphere

#### plus state without decoherence
rotating around the z axis

In [7]:
system = AtomCavitySystem(N=10, g=0, w0=(1.0 * 2 * np.pi),
                              gamma=0, kappa=0, beta=0)

# Build Hamiltonian
H = system.create_hamiltonian()

# Initial state |0⟩_cavity ⊗ |+⟩_atom
psi11 = system.create_initial_state('0', '+')

# Collapse operators
c_ops = system.create_collapse_operators(leaking=False, decay=False, dephasing=False)

# Expectation values
e_ops = system.create_expectation_values(
    number_atom=True,
    number_cavity=True,
    coherence_ge=True,
    coherence_eg=True
)

# Time evolution
times = np.linspace(0, 20, 500)
result = qt.mesolve(H, psi11, times, c_ops, e_ops, options=qt.Options(store_states=True))

#system.create_bloch_gif(result, "plus_state.gif")

#### plus state with decoherence

In [8]:
system = AtomCavitySystem(N=10, g=0, w0=(1.0 * 2 * np.pi),
                              gamma=0, kappa=0, beta=0.1)

# Build Hamiltonian
H = system.create_hamiltonian()

# Initial state |0⟩_cavity ⊗ |+⟩_atom
psi11 = system.create_initial_state('0', '+')

# Collapse operators
c_ops = system.create_collapse_operators(leaking=False, decay=False, dephasing=True)

# Expectation values
e_ops = system.create_expectation_values(
    number_atom=True,
    number_cavity=True,
    coherence_ge=True,
    coherence_eg=True
)

# Time evolution
times = np.linspace(0, 20, 500)
result = qt.mesolve(H, psi11, times, c_ops, e_ops, options=qt.Options(store_states=True))

#system.create_bloch_gif(result, "plus_state_dec.gif")

#### plus state with echo

In [9]:
X_pi = qt.tensor(qt.qeye(system.N), qt.sigmax())

system = AtomCavitySystem(N=10, g=0, w0=(1.0 * 2 * np.pi),
                          gamma=0, kappa=0, beta=0.03)

# Build Hamiltonian
H = system.create_hamiltonian()

# Initial state |1⟩_cavity ⊗ |+⟩_atom
psi0 = system.create_initial_state('1', '+')

# Collapse operators
c_ops = system.create_collapse_operators(leaking=True, decay=True, dephasing=True)

# Expectation values
e_ops = system.create_expectation_values(
    number_atom=True,
    number_cavity=True,
    coherence_ge=True,
    coherence_eg=True
)

# Total simulation time
t_max = 20
t_echo = 5.25
times1 = np.linspace(0, t_echo, 100)
times2 = np.linspace(t_echo, t_max, 400)

# First evolution until t_echo
result1 = qt.mesolve(H, psi0, times1, c_ops, e_ops, options=qt.Options(store_states=True))

# Apply π-pulse (σx on atom)
rho_mid = result1.states[-1]
rho_mid_pulsed = X_pi * rho_mid * X_pi.dag()

# Continue evolution after echo
result2 = qt.mesolve(H, rho_mid_pulsed, times2, c_ops, e_ops, options=qt.Options(store_states=True))

# Combine results
times_full = np.concatenate([times1, times2])
all_states = result1.states + result2.states

# Create a new result-like object with states and times
class SimpleResult:
    def __init__(self, states, times):
        self.states = states
        self.times = times

combined_result = SimpleResult(all_states, times_full)

# Create Bloch GIF with smooth interpolation
#system.create_bloch_gif(combined_result, "plus_state_dec_echo.gif")


In [ ]:
# Parameters 

N = 10
g = 0
w0 = 1.0 * 2 * np.pi
gamma = 0.0
kappa = 0.0
beta = 0.0   # do quasi-static noise instead of Lindblad sz!
t_max = 60
t_echo = 10.25
num_realizations = 50
sigma = 0.2   # quasi-static dephasing strength

times1 = np.linspace(0, t_echo, 100)
times2 = np.linspace(t_echo, t_max, 400)
times_full = np.concatenate([times1, times2])

# Pi pulse on atom
X_pi = qt.tensor(qt.qeye(N), qt.sigmax())


# System setup
system = AtomCavitySystem(N=N, g=g, w0=w0, gamma=gamma, kappa=kappa, beta=beta)
H = system.create_hamiltonian()
psi0 = system.create_initial_state('1', '+')

# Expectation values for Bloch vector
e_ops = [system.coh_ge, system.coh_eg]


# Evolve with quasi-static noise
def evolve_with_static_noise(H, psi0, times1, times2, delta_phi):
    H_noise = H + 0.5 * delta_phi * qt.tensor(qt.qeye(N), qt.sigmaz())

    result1 = qt.mesolve(H_noise, psi0, times1, [], e_ops, options=qt.Options(store_states=True))
    rho_mid = result1.states[-1]
    rho_mid_pulsed = X_pi * rho_mid * X_pi.dag()
    result2 = qt.mesolve(H_noise, rho_mid_pulsed, times2, [], e_ops, options=qt.Options(store_states=True))

    result2.states = result1.states + result2.states
    result2.times = np.concatenate([times1, times2])
    return result2

# Run multiple realizations
all_states = []
for _ in range(num_realizations):
    delta_phi = np.random.normal(0, sigma)
    result = evolve_with_static_noise(H, psi0, times1, times2, delta_phi)
    all_states.append(result.states)

# Average over realizations
avg_states = []
for i in range(len(times_full)):
    rho_avg = sum([all_states[r][i] for r in range(num_realizations)]) / num_realizations
    avg_states.append(rho_avg)


# Create GIF
#system.create_bloch_gif(type('Result', (), {'states': avg_states, 'times': times_full})(), 
                        #"plus_state_spin_echo.gif")


Animation saved as plus_state_spin_echo.gif
